In [4]:
import torch
import torch.nn as nn

In [12]:
class FCN_8s(nn.Module):
    def __init__(self):
        super(FCN_8s, self).__init__()
        #conv1
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size = (3,3), stride = (1,1), padding = 1)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size = (3,3), stride = (1,1), padding = 1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2), padding = (1,1))
        #conv2
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size = (2,2), stride = (1,1), padding = (1,1))
        #conv3
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2), padding = 1)
        #conv4
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2), padding = (1,1))
        #conv5
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=(3,3), stride = (1,1), padding = 1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2), padding = (1,1))
        #fully conv
        self.conv6 = nn.Conv2d(512, 4096, kernel_size=(7,7), stride = (2,2), padding = 0)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout(p = 0.5, inplace = True)
        self.conv7 = nn.Conv2d(4096, 4096, kernel_size=(1,1), stride = (2,2), padding = 0)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout(p = 0.5, inplace = True)
        self.score1 = nn.Conv2d(4096, 21, kernel_size = (1,1), stride = (2,2), padding = 0)
        #upsampling
        self.x2upsamp = nn.ConvTranspose2d(21, 21, kernel_size = (4,4), stride = (2,2))
        self.x8upsamp = nn.ConvTranspose2d(21, 21, kernel_size = (16,16), stride = (8,8))
        #pool3,4 conv
        self.pool3conv = nn.Conv2d(256, 21, kernel_size = (1,1))
        self.pool4conv = nn.Conv2d(512, 21, kernel_size = (1,1))
        
    def forward(self, x):
        x = self.relu1_1(self.conv1_1(x))
        x = self.relu1_2(self.conv1_2(x))
        x = self.pool1(x)
        
        x = self.relu2_1(self.conv2_1(x))
        x = self.relu2_2(self.conv2_2(x))
        x = self.pool2(x)
        
        x = self.relu3_1(self.conv3_1(x))
        x = self.relu3_2(self.conv3_2(x))
        x = self.relu3_3(self.conv3_3(x))
        x = self.pool3(x)
        pool3 = x
        
        x = self.relu4_1(self.conv4_1(x))
        x = self.relu4_2(self.conv4_2(x))
        x = self.relu4_3(self.conv4_3(x))
        x = self.pool4(x)
        pool4 = x
        
        x = self.relu5_1(self.conv5_1(x))
        x = self.relu5_2(self.conv5_2(x))
        x = self.relu5_3(self.conv5_3(x))
        x = self.pool5(x)
        
        x = self.drop6(self.relu6(self.conv6(x)))
        x = self.drop7(self.relu7(self.conv7(x)))
        x = self.score1(x)
        
        x = self.x2upsamp(x)
        
        pool4 = self.pool4conv(pool4)
        
        x = x + pool4
        x = self.x2upsamp(x)
        
        pool3 = self.pool3conv(pool3)
        
        x = x + pool3
        x = self.x8upsamp(x)
        
        return x

In [15]:
model = FCN_8s()
print(model)

FCN_8s(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1_1): ReLU(inplace=True)
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_1): ReLU(inplace=True)
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace=True)
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU(inplace=True)
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU(inplace=True)
  (po

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001, weight_decay = 0.0016, momentum = 0.9)